In [18]:
import pandas as pd

train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [19]:
train = train.drop(["PassengerId", "Name"], axis=1)
test = test.drop(["PassengerId", "Name"], axis=1)

## data preprocessing

In [20]:
train["Cabin"] = train["Cabin"].str.slice(0,1,1)  # 가장 유의미한것으로 보였던 deck 변수만 사용

In [21]:
train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


### Missing data proc

In [44]:
import numpy as np

# Age에 0이 있음 -> 결측치로 판단
# 평균값으로 대체

train["Age"] = train["Age"].replace(0.0, np.NaN)
train["Age"] = train["Age"].fillna(int(train["Age"].mean()))

test["Age"] = test["Age"].replace(0.0, np.NaN)
test["Age"] = test["Age"].fillna(int(test["Age"].mean()))  # test age mean를 구할 땐 train data를 사용해야함?

In [23]:
train["Age"].describe()

count    8693.000000
mean       29.425285
std        13.719843
min         1.000000
25%        20.000000
50%        28.000000
75%        37.000000
max        79.000000
Name: Age, dtype: float64

In [45]:
fill_0 = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

for column in fill_0:
    train[column] = train[column].fillna(0)
    test[column] = test[column].fillna(0)

In [58]:
categorical = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"]

for column in train.columns:
    if column in categorical:
        train[column] = train[column].fillna("None")
        test[column] = test[column].fillna("None")

In [59]:
X_train = train.drop("Transported", axis=1)
y_train = train["Transported"]

X_test = test

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [11]:
X_train[:8]

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
3600,Earth,False,G,TRAPPIST-1e,29.0,False,0.0,0.0,0.0,0.0,0.0
1262,Earth,True,G,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0
8612,Earth,NaN,G,PSO J318.5-22,35.0,False,0.0,0.0,0.0,0.0,0.0
5075,Europa,True,D,55 Cancri e,26.0,False,0.0,0.0,0.0,0.0,0.0
4758,Earth,False,G,TRAPPIST-1e,13.0,False,0.0,0.0,60.0,1.0,5147.0
5363,Earth,False,F,TRAPPIST-1e,24.0,False,0.0,217.0,0.0,77.0,510.0
774,Mars,False,F,TRAPPIST-1e,29.0,False,0.0,0.0,0.0,0.0,0.0
2749,NaN,False,E,55 Cancri e,25.0,False,0.0,641.0,0.0,2.0,0.0


## 모델 생성, 추론

In [57]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=10000, random_state=42, nan_mode="Min")
model.fit(X_train, y_train, cat_features=categorical, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100)

Learning rate set to 0.018791
0:	learn: 0.6839723	test: 0.6837211	best: 0.6837211 (0)	total: 175ms	remaining: 29m 12s
1:	learn: 0.6747776	test: 0.6743264	best: 0.6743264 (1)	total: 224ms	remaining: 18m 40s
2:	learn: 0.6666261	test: 0.6661540	best: 0.6661540 (2)	total: 245ms	remaining: 13m 37s
3:	learn: 0.6576818	test: 0.6571933	best: 0.6571933 (3)	total: 284ms	remaining: 11m 50s
4:	learn: 0.6507755	test: 0.6505589	best: 0.6505589 (4)	total: 310ms	remaining: 10m 19s
5:	learn: 0.6431934	test: 0.6429609	best: 0.6429609 (5)	total: 344ms	remaining: 9m 33s
6:	learn: 0.6361397	test: 0.6356397	best: 0.6356397 (6)	total: 407ms	remaining: 9m 41s
7:	learn: 0.6291460	test: 0.6286698	best: 0.6286698 (7)	total: 439ms	remaining: 9m 8s
8:	learn: 0.6219838	test: 0.6214681	best: 0.6214681 (8)	total: 467ms	remaining: 8m 37s
9:	learn: 0.6168418	test: 0.6162334	best: 0.6162334 (9)	total: 490ms	remaining: 8m 9s
10:	learn: 0.6117877	test: 0.6112355	best: 0.6112355 (10)	total: 511ms	remaining: 7m 44s
11:	lear

93:	learn: 0.4526423	test: 0.4560125	best: 0.4560125 (93)	total: 3.07s	remaining: 5m 23s
94:	learn: 0.4520097	test: 0.4554554	best: 0.4554554 (94)	total: 3.09s	remaining: 5m 22s
95:	learn: 0.4513539	test: 0.4549381	best: 0.4549381 (95)	total: 3.12s	remaining: 5m 21s
96:	learn: 0.4507543	test: 0.4543750	best: 0.4543750 (96)	total: 3.17s	remaining: 5m 23s
97:	learn: 0.4502733	test: 0.4538614	best: 0.4538614 (97)	total: 3.19s	remaining: 5m 22s
98:	learn: 0.4498303	test: 0.4534850	best: 0.4534850 (98)	total: 3.21s	remaining: 5m 21s
99:	learn: 0.4496342	test: 0.4532407	best: 0.4532407 (99)	total: 3.25s	remaining: 5m 21s
100:	learn: 0.4488432	test: 0.4525191	best: 0.4525191 (100)	total: 3.27s	remaining: 5m 21s
101:	learn: 0.4481571	test: 0.4519324	best: 0.4519324 (101)	total: 3.3s	remaining: 5m 20s
102:	learn: 0.4476547	test: 0.4514924	best: 0.4514924 (102)	total: 3.32s	remaining: 5m 19s
103:	learn: 0.4470037	test: 0.4508279	best: 0.4508279 (103)	total: 3.35s	remaining: 5m 18s
104:	learn: 0.

191:	learn: 0.4212141	test: 0.4292784	best: 0.4292784 (191)	total: 6.26s	remaining: 5m 19s
192:	learn: 0.4210382	test: 0.4291744	best: 0.4291744 (192)	total: 6.29s	remaining: 5m 19s
193:	learn: 0.4208729	test: 0.4290107	best: 0.4290107 (193)	total: 6.31s	remaining: 5m 19s
194:	learn: 0.4207251	test: 0.4288414	best: 0.4288414 (194)	total: 6.33s	remaining: 5m 18s
195:	learn: 0.4205119	test: 0.4288119	best: 0.4288119 (195)	total: 6.36s	remaining: 5m 17s
196:	learn: 0.4204540	test: 0.4287593	best: 0.4287593 (196)	total: 6.38s	remaining: 5m 17s
197:	learn: 0.4202472	test: 0.4286152	best: 0.4286152 (197)	total: 6.4s	remaining: 5m 16s
198:	learn: 0.4202294	test: 0.4285970	best: 0.4285970 (198)	total: 6.41s	remaining: 5m 15s
199:	learn: 0.4200572	test: 0.4284213	best: 0.4284213 (199)	total: 6.44s	remaining: 5m 15s
200:	learn: 0.4198284	test: 0.4282797	best: 0.4282797 (200)	total: 6.49s	remaining: 5m 16s
201:	learn: 0.4196687	test: 0.4281906	best: 0.4281906 (201)	total: 6.53s	remaining: 5m 16s


284:	learn: 0.4098410	test: 0.4207056	best: 0.4207056 (284)	total: 8.88s	remaining: 5m 2s
285:	learn: 0.4098165	test: 0.4206945	best: 0.4206945 (285)	total: 8.89s	remaining: 5m 1s
286:	learn: 0.4096844	test: 0.4206665	best: 0.4206665 (286)	total: 8.92s	remaining: 5m 1s
287:	learn: 0.4095894	test: 0.4206345	best: 0.4206345 (287)	total: 8.94s	remaining: 5m 1s
288:	learn: 0.4095118	test: 0.4206047	best: 0.4206047 (288)	total: 8.98s	remaining: 5m 1s
289:	learn: 0.4094401	test: 0.4205856	best: 0.4205856 (289)	total: 9s	remaining: 5m 1s
290:	learn: 0.4093928	test: 0.4205373	best: 0.4205373 (290)	total: 9.03s	remaining: 5m 1s
291:	learn: 0.4093766	test: 0.4205278	best: 0.4205278 (291)	total: 9.07s	remaining: 5m 1s
292:	learn: 0.4092777	test: 0.4205041	best: 0.4205041 (292)	total: 9.13s	remaining: 5m 2s
293:	learn: 0.4091972	test: 0.4204983	best: 0.4204983 (293)	total: 9.15s	remaining: 5m 2s
294:	learn: 0.4091887	test: 0.4204857	best: 0.4204857 (294)	total: 9.17s	remaining: 5m 1s
295:	learn: 0

376:	learn: 0.4031029	test: 0.4166753	best: 0.4166753 (376)	total: 11.5s	remaining: 4m 53s
377:	learn: 0.4029717	test: 0.4166161	best: 0.4166161 (377)	total: 11.6s	remaining: 4m 54s
378:	learn: 0.4029299	test: 0.4165541	best: 0.4165541 (378)	total: 11.6s	remaining: 4m 54s
379:	learn: 0.4027697	test: 0.4165484	best: 0.4165484 (379)	total: 11.6s	remaining: 4m 54s
380:	learn: 0.4026929	test: 0.4165055	best: 0.4165055 (380)	total: 11.7s	remaining: 4m 55s
381:	learn: 0.4026733	test: 0.4164903	best: 0.4164903 (381)	total: 11.7s	remaining: 4m 54s
382:	learn: 0.4025455	test: 0.4164514	best: 0.4164514 (382)	total: 11.7s	remaining: 4m 54s
383:	learn: 0.4024519	test: 0.4163856	best: 0.4163856 (383)	total: 11.8s	remaining: 4m 54s
384:	learn: 0.4023530	test: 0.4163982	best: 0.4163856 (383)	total: 11.8s	remaining: 4m 54s
385:	learn: 0.4023429	test: 0.4163917	best: 0.4163856 (383)	total: 11.8s	remaining: 4m 53s
386:	learn: 0.4022695	test: 0.4163628	best: 0.4163628 (386)	total: 11.8s	remaining: 4m 53s

...

1033:	learn: 0.3587315	test: 0.4067641	best: 0.4067641 (1033)	total: 31s	remaining: 4m 28s
1034:	learn: 0.3586807	test: 0.4067680	best: 0.4067641 (1033)	total: 31s	remaining: 4m 28s
1035:	learn: 0.3586050	test: 0.4067059	best: 0.4067059 (1035)	total: 31.1s	remaining: 4m 28s
1036:	learn: 0.3585113	test: 0.4067220	best: 0.4067059 (1035)	total: 31.1s	remaining: 4m 28s
1037:	learn: 0.3584695	test: 0.4067047	best: 0.4067047 (1037)	total: 31.1s	remaining: 4m 28s
1038:	learn: 0.3583768	test: 0.4066921	best: 0.4066921 (1038)	total: 31.2s	remaining: 4m 28s
1039:	learn: 0.3583018	test: 0.4066530	best: 0.4066530 (1039)	total: 31.2s	remaining: 4m 29s
1040:	learn: 0.3582758	test: 0.4066308	best: 0.4066308 (1040)	total: 31.3s	remaining: 4m 29s
1041:	learn: 0.3581706	test: 0.4066581	best: 0.4066308 (1040)	total: 31.3s	remaining: 4m 28s
1042:	learn: 0.3581380	test: 0.4066826	best: 0.4066308 (1040)	total: 31.3s	remaining: 4m 28s
1043:	learn: 0.3580908	test: 0.4067412	best: 0.4066308 (1040)	total: 31.4s

1127:	learn: 0.3537926	test: 0.4065689	best: 0.4065656 (1126)	total: 36.5s	remaining: 4m 47s
1128:	learn: 0.3537155	test: 0.4065759	best: 0.4065656 (1126)	total: 36.6s	remaining: 4m 47s
1129:	learn: 0.3536400	test: 0.4065882	best: 0.4065656 (1126)	total: 36.6s	remaining: 4m 47s
1130:	learn: 0.3536108	test: 0.4066267	best: 0.4065656 (1126)	total: 36.7s	remaining: 4m 47s
1131:	learn: 0.3535550	test: 0.4066107	best: 0.4065656 (1126)	total: 36.7s	remaining: 4m 47s
1132:	learn: 0.3535261	test: 0.4065916	best: 0.4065656 (1126)	total: 36.8s	remaining: 4m 48s
1133:	learn: 0.3534809	test: 0.4065906	best: 0.4065656 (1126)	total: 36.8s	remaining: 4m 47s
1134:	learn: 0.3534093	test: 0.4066103	best: 0.4065656 (1126)	total: 36.9s	remaining: 4m 47s
1135:	learn: 0.3533838	test: 0.4066052	best: 0.4065656 (1126)	total: 36.9s	remaining: 4m 47s
1136:	learn: 0.3533610	test: 0.4065949	best: 0.4065656 (1126)	total: 36.9s	remaining: 4m 47s
1137:	learn: 0.3533346	test: 0.4065931	best: 0.4065656 (1126)	total: 3

1217:	learn: 0.3492578	test: 0.4058567	best: 0.4058530 (1215)	total: 42s	remaining: 5m 3s
1218:	learn: 0.3492271	test: 0.4058595	best: 0.4058530 (1215)	total: 42.1s	remaining: 5m 3s
1219:	learn: 0.3490790	test: 0.4058435	best: 0.4058435 (1219)	total: 42.1s	remaining: 5m 3s
1220:	learn: 0.3490352	test: 0.4058564	best: 0.4058435 (1219)	total: 42.2s	remaining: 5m 3s
1221:	learn: 0.3489668	test: 0.4058580	best: 0.4058435 (1219)	total: 42.3s	remaining: 5m 4s
1222:	learn: 0.3489568	test: 0.4058323	best: 0.4058323 (1222)	total: 42.4s	remaining: 5m 4s
1223:	learn: 0.3489249	test: 0.4058235	best: 0.4058235 (1223)	total: 42.5s	remaining: 5m 4s
1224:	learn: 0.3488078	test: 0.4058068	best: 0.4058068 (1224)	total: 42.6s	remaining: 5m 5s
1225:	learn: 0.3487320	test: 0.4057884	best: 0.4057884 (1225)	total: 42.6s	remaining: 5m 5s
1226:	learn: 0.3487036	test: 0.4057856	best: 0.4057856 (1226)	total: 42.7s	remaining: 5m 5s
1227:	learn: 0.3486767	test: 0.4057888	best: 0.4057856 (1226)	total: 42.7s	remaini

1306:	learn: 0.3440943	test: 0.4055016	best: 0.4054342 (1291)	total: 47.7s	remaining: 5m 17s
1307:	learn: 0.3440560	test: 0.4055123	best: 0.4054342 (1291)	total: 47.7s	remaining: 5m 17s
1308:	learn: 0.3440310	test: 0.4055040	best: 0.4054342 (1291)	total: 47.8s	remaining: 5m 17s
1309:	learn: 0.3439448	test: 0.4054918	best: 0.4054342 (1291)	total: 47.9s	remaining: 5m 17s
1310:	learn: 0.3439182	test: 0.4055203	best: 0.4054342 (1291)	total: 47.9s	remaining: 5m 17s
1311:	learn: 0.3438841	test: 0.4055360	best: 0.4054342 (1291)	total: 48s	remaining: 5m 17s
1312:	learn: 0.3438404	test: 0.4055541	best: 0.4054342 (1291)	total: 48s	remaining: 5m 17s
1313:	learn: 0.3437801	test: 0.4055022	best: 0.4054342 (1291)	total: 48.1s	remaining: 5m 17s
1314:	learn: 0.3437656	test: 0.4054872	best: 0.4054342 (1291)	total: 48.1s	remaining: 5m 17s
1315:	learn: 0.3436840	test: 0.4055088	best: 0.4054342 (1291)	total: 48.1s	remaining: 5m 17s
1316:	learn: 0.3436428	test: 0.4055047	best: 0.4054342 (1291)	total: 48.2s

1397:	learn: 0.3397557	test: 0.4054510	best: 0.4053711 (1334)	total: 53.3s	remaining: 5m 27s
1398:	learn: 0.3396810	test: 0.4054691	best: 0.4053711 (1334)	total: 53.3s	remaining: 5m 27s
1399:	learn: 0.3396655	test: 0.4054654	best: 0.4053711 (1334)	total: 53.3s	remaining: 5m 27s
1400:	learn: 0.3396364	test: 0.4054781	best: 0.4053711 (1334)	total: 53.4s	remaining: 5m 27s
1401:	learn: 0.3395666	test: 0.4054652	best: 0.4053711 (1334)	total: 53.5s	remaining: 5m 28s
1402:	learn: 0.3395448	test: 0.4055371	best: 0.4053711 (1334)	total: 53.6s	remaining: 5m 28s
1403:	learn: 0.3394662	test: 0.4055337	best: 0.4053711 (1334)	total: 53.7s	remaining: 5m 28s
1404:	learn: 0.3394324	test: 0.4055494	best: 0.4053711 (1334)	total: 53.7s	remaining: 5m 28s
1405:	learn: 0.3393975	test: 0.4055441	best: 0.4053711 (1334)	total: 53.8s	remaining: 5m 28s
1406:	learn: 0.3393837	test: 0.4055464	best: 0.4053711 (1334)	total: 53.9s	remaining: 5m 28s
1407:	learn: 0.3393314	test: 0.4055001	best: 0.4053711 (1334)	total: 5

In [60]:
pred = model.predict(X_test)

In [61]:
pred

array(['True', 'False', 'True', ..., 'True', 'True', 'True'], dtype=object)

In [64]:
submission["Transported"] = pred

In [65]:
submission.to_csv("submission.csv",index=False)